Mount to gdrive

In [ ]:
#do not run if you workspace is located somewhere else
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks")
os.listdir()
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

Step 1 ################################################################################

In [ ]:
#imports and installs required packages and libraries
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts.xml")

!pip install python-terrier
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

import pandas as pd
!rm -rf ./pd_index
pd_indexer = pt.DFIndexer("./pd_index")

import nltk
nltk.download('punkt')
nltk.download('stopwords')

Index Documents

In [ ]:
docno_list = []
url_list = []
text_list = []
#indexed questions
print("Indexing Questions...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  title = question.title.lower().strip()
  body = question.body.lower().strip()
  text = title + " " + body
  docno_list.append(str(question_id))
  url_list.append("url"+str(question_id))
  text_list.append(text)
print("Complete")

#indexes answers
print("Indexing Answers...")
for answer_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[answer_id]
  text = answer.body.lower().strip()
  docno_list.append(str(answer_id))
  url_list.append("url"+str(answer_id))
  text_list.append(text)
print("Complete")
print("Finalizing...")

#adds all collected data to a DataFrame from above
df = pd.DataFrame({'docno':docno_list, 'url':url_list, 'text':text_list})
indexref = pd_indexer.index(df["text"], df)
index = pt.IndexFactory.of(indexref)

#prints info on colected documents, (should have 185140 documents)
print(index.getCollectionStatistics().toString())

Indexing Questions...
Complete
Indexing Answers...
Complete
Finalizing...
Number of documents: 185140
Number of terms: 83738
Number of postings: 6613484
Number of fields: 0
Number of tokens: 10959038
Field names: []
Positions:   false



Searches using TF_IDF

In [ ]:
#add querys to dataframe
queries = pd.DataFrame([["1", "weathering effects"], ["2", "fix shading problem"], ["3","Nurbs path doesnt go fully closed"], ["4","Problem with bone"], ["5","Scaling faces with drivers"], 
                       ["6","Glowing text effect"], ["7","Creating a hole out of an custom shape"], ["8","Brick texture with multiple colors"], ["9","How to Dribble soccer ball"], ["10","Masking selected vertices for sculpting"], 
                       ["11","How to use Modular Trees"], ["12","Netrender addon missing"], ["13","drawing a belt around a gear"], ["14","Constrain vertex positions to moving objects"], ["15","Deformation of particle objects"], 
                       ["16","change smoke color"], ["17","Mix shader is not working"], ["18","Scale deformation when rotating imported FBX"], ["19","cubic trefoil knot"], ["20","How to get the absolute path"]],columns=['qid','query'])
#find top 1000 results with pyterrier Batch Retrieve, from the created index
result = pt.BatchRetrieve(indexref, num_results =1000, wmodel="TF_IDF").transform(queries)
#write results to file and print
pt.io.write_results(result, "TF_IDF_result.txt", format='trec')
print(result)


      qid   docid   docno  rank      score                         query
0       1   50417  146393     0  15.362622            weathering effects
1       1   22918   69658     1  14.222676            weathering effects
2       1  125832   80137     2  11.577481            weathering effects
3       1   69376  197276     3  10.520627            weathering effects
4       1   83797  237503     4   9.948666            weathering effects
...    ..     ...     ...   ...        ...                           ...
19275  20    3032   10619   995   4.378565  How to get the absolute path
19276  20   86668  243588   996   4.378565  How to get the absolute path
19277  20   99734    7952   997   4.376581  How to get the absolute path
19278  20   91119  254069   998   4.369923  How to get the absolute path
19279  20    4401   16241   999   4.369399  How to get the absolute path

[19280 rows x 6 columns]


Searches using BM25

In [ ]:
#add querys to dataframe
queries = pd.DataFrame([["1", "weathering effects"], ["2", "fix shading problem"], ["3","Nurbs path doesnt go fully closed"], ["4","Problem with bone"], ["5","Scaling faces with drivers"], 
                       ["6","Glowing text effect"], ["7","Creating a hole out of an custom shape"], ["8","Brick texture with multiple colors"], ["9","How to Dribble soccer ball"], ["10","Masking selected vertices for sculpting"], 
                       ["11","How to use Modular Trees"], ["12","Netrender addon missing"], ["13","drawing a belt around a gear"], ["14","Constrain vertex positions to moving objects"], ["15","Deformation of particle objects"], 
                       ["16","change smoke color"], ["17","Mix shader is not working"], ["18","Scale deformation when rotating imported FBX"], ["19","cubic trefoil knot"], ["20","How to get the absolute path"]],columns=['qid','query'])
#find top 1000 results with pyterrier Batch Retrieve, from the created index
result = pt.BatchRetrieve(indexref, num_results =1000, wmodel="BM25").transform(queries)
#write results to file and print
pt.io.write_results(result, "BM25_result.txt", format='trec')
print(result)

      qid   docid   docno  rank      score                         query
0       1   50417  146393     0  27.872407            weathering effects
1       1   22918   69658     1  25.788016            weathering effects
2       1  125832   80137     2  20.991848            weathering effects
3       1   69376  197276     3  19.075601            weathering effects
4       1   83797  237503     4  17.995605            weathering effects
...    ..     ...     ...   ...        ...                           ...
19275  20  117432   57949   995   7.854021  How to get the absolute path
19276  20  176960  241453   996   7.849800  How to get the absolute path
19277  20    3487   13781   997   7.846381  How to get the absolute path
19278  20   21376   65801   998   7.846381  How to get the absolute path
19279  20   39663  116366   999   7.846381  How to get the absolute path

[19280 rows x 6 columns]


Searches using LGD

In [ ]:
#add querys to dataframe
queries = pd.DataFrame([["1", "weathering effects"], ["2", "fix shading problem"], ["3","Nurbs path doesnt go fully closed"], ["4","Problem with bone"], ["5","Scaling faces with drivers"], 
                       ["6","Glowing text effect"], ["7","Creating a hole out of an custom shape"], ["8","Brick texture with multiple colors"], ["9","How to Dribble soccer ball"], ["10","Masking selected vertices for sculpting"], 
                       ["11","How to use Modular Trees"], ["12","Netrender addon missing"], ["13","drawing a belt around a gear"], ["14","Constrain vertex positions to moving objects"], ["15","Deformation of particle objects"], 
                       ["16","change smoke color"], ["17","Mix shader is not working"], ["18","Scale deformation when rotating imported FBX"], ["19","cubic trefoil knot"], ["20","How to get the absolute path"]],columns=['qid','query'])
#find top 1000 results with pyterrier Batch Retrieve, from the created index
result = pt.BatchRetrieve(indexref, num_results =1000, wmodel="LGD").transform(queries)
#write results to file and print
pt.io.write_results(result, "LGD_result.txt", format='trec')
print(result)

      qid   docid   docno  rank      score                         query
0       1   50417  146393     0  20.209381            weathering effects
1       1   22918   69658     1  19.495530            weathering effects
2       1  125832   80137     2  17.713862            weathering effects
3       1   83797  237503     3  17.210715            weathering effects
4       1   51917  150082     4  17.101264            weathering effects
...    ..     ...     ...   ...        ...                           ...
19275  20  166565  206249   995   7.075675  How to get the absolute path
19276  20  184844  264882   996   7.075675  How to get the absolute path
19277  20   97213    2067   997   7.075295  How to get the absolute path
19278  20   23819   72069   998   7.070720  How to get the absolute path
19279  20  139164  120411   999   7.070183  How to get the absolute path

[19280 rows x 6 columns]


Step 2: Fusion################################################################################

In [ ]:
!pip install ranx
from ranx import fuse, evaluate, Qrels, Run

In [ ]:
#uses created run and qrel files to find precision and NDCG wsing evaluate
run_1 = Run.from_file("TF_IDF_result_5.txt", kind="trec")
run_2 = Run.from_file("BM25_result_5.txt", kind="trec")
qrels = Qrels.from_file("qrel.txt", kind="trec")
combined_run = fuse(runs=[run_1, run_2],norm="min-max", method="mnz")

#print fused results
print("fused NDCG: ", evaluate(qrels, combined_run, "ndcg@100"))
print("fused precision: ", evaluate(qrels, combined_run, "precision@5-l2"))
print()
#print bm25 results
print("bm25 NDCG: ", evaluate(qrels, run_2, "ndcg@100"))
print("bm25 precision: ", evaluate(qrels, run_2, "precision@5-l2"))
print()
#print TF-IDF results
print("tf-idf NDCG: ", evaluate(qrels, run_1, "ndcg@100"))
print("tf-idf precision: ", evaluate(qrels, run_1, "precision@5-l2"))

combined_run.save("combMNZ.tsv", kind="trec")

fused NDCG:  0.8748736380425195
fused precision:  0.4499999999999999

bm25 NDCG:  0.8512408347702264
bm25 precision:  0.43

tf-idf NDCG:  0.8584844891445419
tf-idf precision:  0.4499999999999999


Step 3 Weighted Combination(Optional): Fusion################################################################

Step 4: Query Expansion################################################################################

In [ ]:
pipeline_RM3 = (pt.BatchRetrieve(indexref, num_results =1000, wmodel="BM25") >>
        pt.rewrite.RM3(indexref) >>
        pt.BatchRetrieve(indexref, num_results =1000, wmodel="BM25")
)
bm25_RM3_Results = pipeline_RM3.transform(queries)
pt.io.write_results(bm25_RM3_Results, "RM3_result.txt", format='trec')
print(bm25_RM3_Results)

      qid   docid   docno  rank      score                     query_0  \
0       1   50417  146393     0  27.612747          weathering effects   
1       1   22918   69658     1  27.224355          weathering effects   
2       1  125832   80137     2  23.891261          weathering effects   
3       1   69376  197276     3  18.885684          weathering effects   
4       1   83797  237503     4  17.772629          weathering effects   
...    ..     ...     ...   ...        ...                         ...   
19995   9  143999  134538   995   6.906847  How to Dribble soccer ball   
19996   9  173201  228550   996   6.906847  How to Dribble soccer ball   
19997   9  125949   80457   997   6.900682  How to Dribble soccer ball   
19998   9   20447   63513   998   6.861258  How to Dribble soccer ball   
19999   9   52248  150875   999   6.861258  How to Dribble soccer ball   

                                                   query  
0      applypipeline:off sure^0.018255575 densiti^0.

In [ ]:
run_3 = Run.from_file("RM3_result.txt", kind="trec")
print(run_3.name, evaluate(qrels, run_3, "ndcg@100"))
print("RM3-BM25 precision: %.3f"%evaluate(qrels, run_3, "precision@5-l2"))

pyterrier 0.8832130606769694
RM3-BM25 precision: 0.390


Step 5: Evaluation and Comparison##################################################################

In [ ]:
#uses created run and qrel files to find precision and NDCG wsing evaluate
tfidf_run = Run.from_file("TF_IDF_result_5.txt", kind="trec")
bm25_run = Run.from_file("BM25_result_5.txt", kind="trec")
lgd_run = Run.from_file("LGD_result_5.txt", kind="trec")
qrel = Qrels.from_file("qrel.txt", kind="trec")

#print TF-IDF results
print("TF-IDF precision: %.3f"%evaluate(qrel, tfidf_run, "precision@5-l2"))
print("TF-IDF NDCG %.3f"%evaluate(qrel, tfidf_run, "ndcg@100"))
print()
#print BM25 results
print("BM25 precision: %.3f"%evaluate(qrel, bm25_run, "precision@5-l2"))
print("BM25 NDCG %.3f"%evaluate(qrel, bm25_run, "ndcg@100"))
print()
#print LGD results
print("LGD precision: %.3f"%evaluate(qrel, lgd_run, "precision@5-l2"))
print("LGD NDCG %.3f"%evaluate(qrel, lgd_run, "ndcg@100"))

TF-IDF precision: 0.450
TF-IDF NDCG 0.858

BM25 precision: 0.430
BM25 NDCG 0.851

LGD precision: 0.490
LGD NDCG 0.871
